In [ ]:
# B.Tech Project, Indian Insititute of Technology, Delhi
# Authors - Harshita (2019CS190272), Om Krishna (2019CS10272)
# Input : Lineament, Lithology, Drainage density rasters

# !pip install rasterio
import requests
import rasterio as rio
from rasterio.enums import Resampling
from rasterio.plot import show
import numpy as np
from matplotlib import pyplot
from PIL import Image
from osgeo import gdal
import os.path
import time
import math
from tqdm import tqdm

In [ ]:
# input files
folder = 'jamui_rp_files/new/'
lineaments = folder + 'new_lineaments.tif'
lithology = folder + 'geo_litho.tif'
drainage = folder + 'new_dd.tif'
sloperc = folder + 'jamui_sloperc.tif'

In [ ]:
# combined using the best resolution

lin_r = rio.open(lineaments)
lith_r = rio.open(lithology)
drain_r = rio.open(drainage)

lin = lin_r.read(1).astype(float)
lith = lith_r.read(1).astype(float)
drain = drain_r.read(1).astype(float)

m1, n1 = lin.shape
m2, n2 = lith.shape
m3, n3 = drain.shape

print(m1, n1)
print(m2, n2)
print(m3, n3)

# slope
slope_r = rio.open(sloperc)
slope = slope_r.read(1).astype(float)

m4, n4 = slope.shape
print(m4, n4)

maxi = np.amax(slope)

# best resolution
m = max(m1,m2,m3)
n = max(n1,n2,n3)

In [ ]:
# Calculating Recharge Potentiality
idef = '_final'

rp = np.zeros((m,n))
rptif = folder + 'rp_newgeo' + idef + '.tif'

for ic in tqdm(range(0,n)):
    for ir in range (0,m):
        
        # index i loops through largest raster, j and k through smaller rasters
        # j and k indices obtained by below rules
        jc = (n1 * ic) // n
        jr = (m1 * ir) // m
        kc = (n2 * ic) // n
        kr = (m2 * ir) // m
        
        rp[ir][ic] = drain[ir][ic] * lin[jr][jc] * lith[kr][kc]
        
        # recharge potentiality classification
        # 1 is high RP, 2 is moderate, 3 is low
        if rp[ir][ic] == 1 or rp[ir][ic] == 2 or rp[ir][ic] == 10 or rp[ir][ic] == 20 or rp[ir][ic] == 30:
            rp[ir][ic] = 1
        elif rp[ir][ic] == 3 or rp[ir][ic] == 4:
            rp[ir][ic] = 2
        elif rp[ir][ic] == 6 or rp[ir][ic] == 9:
            rp[ir][ic] = 3
        else:
            rp[ir][ic] = 0

out_tiff = Image.fromarray(rp)
out_tiff.save(rptif)

# georeference jamui
georeffed = folder + 'geo_rp_newgeo' + idef + '.tif'
while not os.path.exists(rptif):
    time.sleep(1)
ds = gdal.Open(rptif)

options_list = [
    '-of GTiff',
    '-a_srs EPSG:4326',
    # '-a_ullr 83.323 27.521 88.298 24.286'  #bihar
    '-a_ullr 85.8201828002929972 25.1442337036132990 86.6115875244140057 24.3684406280518004' #jamui
] 

options_string = " ".join(options_list)

ds = gdal.Translate(georeffed,
                    rptif,
                    options=options_string)

ds = None

In [ ]:
# Merging Recharge potentiality with Slope
tc = np.zeros((m,n))
tctif = folder + 'tc_newgeo' + idef + '.tif'

for ic in tqdm(range(0,n)):
    for ir in range (0,m):
        jc = (n4 * ic) // n
        jr = (m4 * ir) // m
         
        r = rp[ir][ic]
        s = slope[jr][jc]
        
        if s > 0:
            if s <= 0.05 * maxi:
                if r == 1:
                    tc[ir][ic] = 3
                if r == 2:
                    tc[ir][ic] = 2
                if r == 3:
                    tc[ir][ic] = 1
            elif s <= 0.25 * maxi:
                if r == 3 or r == 2:
                    tc[ir][ic] = 4
            elif s > 0.25 * maxi:
                tc[ir][ic] = 5
            else:
                tc[ir][ic] = 0
        else:
            tc[ir][ic] = 0
            

out_tiff = Image.fromarray(tc)
out_tiff.save(tctif)

# georeff jamui
georeffed = folder + 'geo_tc_newgeo' + idef + '.tif'
while not os.path.exists(tctif):
    time.sleep(1)
ds = gdal.Open(tctif)

options_list = [
    '-of GTiff',
    '-a_srs EPSG:4326',
    # '-a_ullr 83.323 27.521 88.298 24.286'  #bihar
    '-a_ullr 85.8201828002929972 25.1442337036132990 86.6115875244140057 24.3684406280518004' #jamui
] 

options_string = " ".join(options_list)

ds = gdal.Translate(georeffed,
                    tctif,
                    options=options_string)

ds = None